In [ ]:
import sys
!{sys.executable} -m pip install notebook ipython-sql SQLAlchemy duckdb-engine

In [2]:
import duckdb
import pandas as pd
#import sqlalchemy
# No need to import duckdb_engine
#  SQLAlchemy will auto-detect the driver needed based on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
%sql duckdb:///data/f1.duckdb

In [9]:
%%sql
select * from information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,None,stage_ergast,drivers,BASE TABLE,None,None,None,None,None,YES,NO,None
1,None,stage_ergast,circuits,BASE TABLE,None,None,None,None,None,YES,NO,None
2,None,stage_ergast,seasons,BASE TABLE,None,None,None,None,None,YES,NO,None
3,None,stage_ergast,constructors,BASE TABLE,None,None,None,None,None,YES,NO,None
4,None,stage_ergast,races,BASE TABLE,None,None,None,None,None,YES,NO,None
5,None,stage_ergast,qualifying,BASE TABLE,None,None,None,None,None,YES,NO,None
6,None,stage_ergast,results,BASE TABLE,None,None,None,None,None,YES,NO,None
7,None,stage_ergast,laps,BASE TABLE,None,None,None,None,None,YES,NO,None
8,None,core,f_laps,BASE TABLE,None,None,None,None,None,YES,NO,None
9,None,core,d_drivers,BASE TABLE,None,None,None,None,None,YES,NO,None


In [25]:
%%sql
SELECT f.race_date
    , drac.season
    , drac.round
    , drac.name AS race_name
    , dcir.country AS circuit_country
    , ddri.full_name AS driver_name
    , dcon.name AS constructor_name
    , f.result_position
    , f.points
    , f.fastest_lap
    , f.status
FROM core.f_results AS f
LEFT JOIN core.d_races AS drac ON drac.id = f.race_id
LEFT JOIN core.d_circuits AS dcir ON dcir.id = f.circuit_id
LEFT JOIN core.d_drivers AS ddri ON ddri.id = f.driver_id
LEFT JOIN core.d_constructors AS dcon ON dcon.id = f.constructor_id
ORDER BY f.race_date DESC
    , f.result_position ASC

,race_date,season,round,race_name,circuit_country,driver_name,constructor_name,result_position,points,fastest_lap,status
0,2022-10-23,2022,19,United States Grand Prix,USA,Max Verstappen,Red Bull,1,25.0,False,Finished
1,2022-10-23,2022,19,United States Grand Prix,USA,Lewis Hamilton,Mercedes,2,18.0,False,Finished
2,2022-10-23,2022,19,United States Grand Prix,USA,Charles Leclerc,Ferrari,3,15.0,False,Finished
3,2022-10-23,2022,19,United States Grand Prix,USA,Sergio Pérez,Red Bull,4,12.0,False,Finished
4,2022-10-23,2022,19,United States Grand Prix,USA,George Russell,Mercedes,5,11.0,True,Finished
5,2022-10-23,2022,19,United States Grand Prix,USA,Lando Norris,McLaren,6,8.0,False,Finished
6,2022-10-23,2022,19,United States Grand Prix,USA,Sebastian Vettel,Aston Martin,7,6.0,False,Finished
7,2022-10-23,2022,19,United States Grand Prix,USA,Kevin Magnussen,Haas F1 Team,8,4.0,False,Finished
8,2022-10-23,2022,19,United States Grand Prix,USA,Yuki Tsunoda,AlphaTauri,9,2.0,False,Finished
9,2022-10-23,2022,19,United States Grand Prix,USA,Esteban Ocon,Alpine F1 Team,10,1.0,False,Finished
